In [1]:
import graphlab as gl
import tensorflow as tf
import matplotlib.pyplot as plt

This non-commercial license of GraphLab Create for academic use is assigned to himadri.mishra.cse13@iitbhu.ac.in and will expire on September 21, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1480536700.log


In [2]:
SF = gl.SFrame.read_csv('ISCX_Botnet-Training.csv',verbose=False)
print "Done reading"
SF.head

Done reading


<bound method SFrame.head of Columns:
	No.	int
	Time	float
	Source	str
	Destination	str
	Protocol	str
	Length	int
	Info	str
	DestInation Port	str
	Source Port	str
	Protocols in frame	str
	Differentiated Services Field	str
	IP_Flags	str
	udp_Length	str
	TCP Segment Len	str
	Sequence number	str
	tcp_Flags	str
	Answer RRs	str
	Source GeoIP Country	str
	Source GeoIP Latitude	str
	Source GeoIP Longitude	str
	Destination GeoIP Latitude	str
	Destination GeoIP Longitude	str
	Destination GeoIP Country	str
	Time to live	str
	Next sequence number	str

Rows: 9388270

Data:
+-----+------------+-------------------+-------------+----------+--------+
| No. |    Time    |       Source      | Destination | Protocol | Length |
+-----+------------+-------------------+-------------+----------+--------+
|  1  |    0.0     | CiscoInc_db:19:c3 |  Broadcast  |   ARP    |   60   |
|  2  |  8.982709  | CiscoInc_db:19:c3 |  Broadcast  |   ARP    |   60   |
|  3  | 50.099564  | CiscoInc_db:19:c3 |  Broadcast  |   

In [3]:
SF = SF[(SF['Source Port']!='')&(SF['DestInation Port']!='')]


In [4]:
SF.rename({'DestInation Port':'Destination Port'})

No.,Time,Source,Destination,Protocol,Length,Info,Destination Port
15,162.765245,147.32.84.180,147.32.84.255,NBNS,110,Registration NBSARUMAN<00> ...,137
16,162.765253,147.32.84.180,147.32.84.255,NBNS,110,Registration NBSARUMAN<00> ...,137
17,163.510681,147.32.84.180,147.32.84.255,NBNS,110,Registration NBSARUMAN<00> ...,137
18,163.510692,147.32.84.180,147.32.84.255,NBNS,110,Registration NBSARUMAN<00> ...,137
22,163.929961,147.32.84.180,147.32.80.9,DNS,64,Standard query 0x6f4c Awpad ...,53
23,163.929967,147.32.84.180,147.32.80.9,DNS,64,Standard query 0x6f4c Awpad ...,53
24,163.930586,147.32.80.9,147.32.84.180,DNS,139,Standard query response0x6f4c No such name A ...,1025
25,163.930994,147.32.84.180,147.32.84.255,NBNS,92,Name query NB WPAD.<00>,137
26,163.930999,147.32.84.180,147.32.84.255,NBNS,92,Name query NB WPAD.<00>,137
27,164.262142,147.32.84.180,147.32.84.255,NBNS,110,Registration NBSARUMAN<00> ...,137


In [5]:
SF['tcp_Flags'] = SF['tcp_Flags'].apply(lambda x:int(x,16) if x!='' else 0)

In [6]:
Protocols = sorted(SF['Protocol'].unique())
#print Protocols

## Some analysis for number of packets of each protocol type

IRC
    "192.168.2.112",
    "131.202.243.84",
    "192.168.5.122",
    "198.164.30.2",
    "192.168.2.110",
    "192.168.5.122",
    "192.168.4.118",
    "192.168.5.122",
    "192.168.2.113",
    "192.168.5.122",
    "192.168.1.103",
    "192.168.5.122",
    "192.168.4.120",
    "192.168.5.122",
    "192.168.2.112",
    "192.168.2.110",
    "192.168.2.112",
    "192.168.4.120",
    "192.168.2.112",
    "192.168.1.103",
    "192.168.2.112",
    "192.168.2.113",
    "192.168.2.112",
    "192.168.4.118",
    "192.168.2.112",
    "192.168.2.109",
    "192.168.2.112",
    "192.168.2.105",
    "192.168.1.105",
    "192.168.5.122",

In [7]:
iplist = [
    "147.32.84.180",
    "147.32.84.170",
    "147.32.84.150",
    "147.32.84.140",
    "147.32.84.130",
    "147.32.84.160",
    "10.0.2.15",
    "192.168.106.141",
    "192.168.106.131",
    "172.16.253.130",
    "172.16.253.131",
    "172.16.253.129",
    "172.16.253.240",
    "74.78.117.238",
    "158.65.110.24",
    "192.168.3.35",
    "192.168.3.25",
    "192.168.3.65",
    "172.29.0.116",
    "172.29.0.109",
    "172.16.253.132",
    "192.168.248.165",
    "10.37.130.4",
    "131.202.243.84", 
    "198.164.30.2", "192.168.5.122", 
    "192.168.5.122", 
    "192.168.5.122", "192.168.5.122", "192.168.5.122", 
    "192.168.2.110", 
    "192.168.4.120", "192.168.1.103",
    "192.168.2.113", "192.168.4.118" , "192.168.2.109", "192.168.2.105", "192.168.5.122"]

MasterBot = [ ("192.168.2.112", "131.202.243.84"), ("192.168.5.122", "198.164.30.2"), ("192.168.2.110", "192.168.5.122"),( "192.168.4.118", "192.168.5.122"), ("192.168.2.113", "192.168.5.122"), ("192.168.1.103", "192.168.5.122"), ("192.168.4.120", "192.168.5.122"), ("192.168.2.112", "192.168.2.110"), ("192.168.2.112", "192.168.4.120"), ("192.168.2.112", "192.168.1.103"), ("192.168.2.112", "192.168.2.113"), ("192.168.2.112", "192.168.4.118"), ("192.168.2.112", "192.168.2.109"), ("192.168.2.112", "192.168.2.105"), ("192.168.1.105", "192.168.5.122")]

iplist = gl.SArray(iplist)
iplist = iplist.unique() 
iplist = list(iplist)

def fcheckIP(x):
    if (x['Source'] in iplist) or (x['Destination'] in iplist):
        return 1
    else:
        if ((x['Source'],x['Destination'])  in MasterBot ) or ((x['Destination'],x['Source'])  in MasterBot ) :
            return 1
        else:
            return 0
        

SF['isBot'] = SF[['Source','Destination']].apply(lambda x: fcheckIP(x))

Botnet Labelled packets and their percentage.


In [8]:
temp = len(SF[SF['isBot']==1])
print temp, (temp*1.0)/len(SF)*100

3874036 41.3869521207


# Sorting will help in identifying flow effectively

## Flow Identification Bidirectional

## No More Brainstorming, this is TRUE !!!!!!!!

In [9]:
def flow_id(x):
    if x['Source']>x['Destination']:
        return x['Source']+'-'+x['Destination']+'-'+str(x['Source Port'])+'-'+str(x['Destination Port'])+'-'+x['Protocol']
    else:
        return x['Destination']+'-'+x['Source']+'-'+str(x['Destination Port'])+'-'+str(x['Source Port'])+'-'+x['Protocol']
SF['UFid'] = SF.apply(lambda x:flow_id(x))

#For identifying IOPR
SF['Forward'] = SF.apply(lambda x: 1 if x['Source']>x['Destination'] else 0 )

## function for comparing two different flows based on columns
def compareUF(x,y):
    if x!=y:
        return False
    return True



##Code for logic of Bidirectional flow identification
import pickle 

FlowNo = 0 ##Unique Flow Number for each flow
prev = None
Flow = []     ##Stores all flows in form of list of dictionary 
#cFlow = []    ##Store the current flow (all details)
count = 0
fc = 0
startTime = None   ##Start Time of each flow to implement timeout
SF = SF.sort(['UFid','Time'])
print 'Done Sorting'
for x in SF:
    if count%500000 == 0:
        print 'Running '+str(count)+' Done !'
        
    count = count+1
    
    if prev is None:
        if startTime is None:
            startTime = x['Time']
        Flow.append(FlowNo)
        
        prev = x['UFid']
    elif compareUF(x['UFid'],prev):
        if x['tcp_Flags']&1:
            Flow.append(FlowNo)
            prev = None
            startTime = None
            FlowNo = FlowNo + 1

        elif x['Time']-startTime>=3600:
            FlowNo = FlowNo + 1
            Flow.append(FlowNo)
            prev = None
            startTime = x['Time']
            
        else:
            
            Flow.append(FlowNo)
            prev = x['UFid']

    else:
        FlowNo = FlowNo + 1
        Flow.append(FlowNo)
        prev = x['UFid']
        startTime = x['Time']


print len(gl.SArray(Flow).unique())



SF['Flow'] = gl.SArray(Flow)
temp = SF.groupby('Flow',{
            'Count':gl.aggregate.COUNT()
        })
print len(temp[temp['Count']>1])

Done Sorting
Running 0 Done !
Running 500000 Done !
Running 1000000 Done !
Running 1500000 Done !
Running 2000000 Done !
Running 2500000 Done !
Running 3000000 Done !
Running 3500000 Done !
Running 4000000 Done !
Running 4500000 Done !
Running 5000000 Done !
Running 5500000 Done !
Running 6000000 Done !
Running 6500000 Done !
Running 7000000 Done !
Running 7500000 Done !
Running 8000000 Done !
Running 8500000 Done !
Running 9000000 Done !
504043
404434


In [10]:

##Code for logic of Bidirectional flow identification
import pickle 

FlowNo = 0 ##Unique Flow Number for each flow
prev = None
Flow = []     ##Stores all flows in form of list of dictionary 
#cFlow = []    ##Store the current flow (all details)
count = 0
fc = 0
startTime = None   ##Start Time of each flow to implement timeout
#SF = SF.sort(['UFid','Time'])
print 'Done Sorting'
for x in SF:
    if count%500000 == 0:
        print 'Running '+str(count)+' Done !'
        
    count = count+1
    
    if prev is None:
        if startTime is None:
            startTime = x['Time']
        Flow.append(FlowNo)
        
        prev = x['UFid']
    elif compareUF(x['UFid'],prev):
        if x['tcp_Flags']&1 or x['tcp_Flags']&4:
            Flow.append(FlowNo)
            prev = None
            startTime = None
            FlowNo = FlowNo + 1

        elif x['Time']-startTime>=3600:
            FlowNo = FlowNo + 1
            Flow.append(FlowNo)
            prev = None
            startTime = x['Time']
            
        else:
            
            Flow.append(FlowNo)
            prev = x['UFid']

    else:
        FlowNo = FlowNo + 1
        Flow.append(FlowNo)
        prev = x['UFid']
        startTime = x['Time']


print len(gl.SArray(Flow).unique())



SF['Flow'] = gl.SArray(Flow)
temp = SF.groupby('Flow',{
            'Count':gl.aggregate.COUNT()
        })
print len(temp[temp['Count']>1])

Done Sorting
Running 0 Done !
Running 500000 Done !
Running 1000000 Done !
Running 1500000 Done !
Running 2000000 Done !
Running 2500000 Done !
Running 3000000 Done !
Running 3500000 Done !
Running 4000000 Done !
Running 4500000 Done !
Running 5000000 Done !
Running 5500000 Done !
Running 6000000 Done !
Running 6500000 Done !
Running 7000000 Done !
Running 7500000 Done !
Running 8000000 Done !
Running 8500000 Done !
Running 9000000 Done !
519392
420390


In [34]:
def flow_id(x):
    if x['Source']>x['Destination']:
        return x['Source']+'-'+x['Destination']+'-'+str(x['Source Port'])+'-'+str(x['Destination Port'])+'-'+x['Protocol']
    else:
        return x['Destination']+'-'+x['Source']+'-'+str(x['Destination Port'])+'-'+str(x['Source Port'])+'-'+x['Protocol']
SF['UFid'] = SF.apply(lambda x:flow_id(x))

In [35]:
#For identifying IOPR
SF['Forward'] = SF.apply(lambda x: 1 if x['Source']>x['Destination'] else 0 )

In [36]:
## function for comparing two different flows based on columns
def compareUF(x,y):
    if x!=y:
        return False
    return True

In [37]:
##Code for logic of Bidirectional flow identification
import pickle 

FlowNo = 0 ##Unique Flow Number for each flow
prev = None
Flow = []     ##Stores all flows in form of list of dictionary 
#cFlow = []    ##Store the current flow (all details)
count = 0
fc = 0
startTime = None   ##Start Time of each flow to implement timeout
SF = SF.sort(['UFid','Time'])
print 'Done Sorting'
for x in SF:
    if count%500000 == 0:
        print 'Running '+str(count)+' Done !'
        
    count = count+1
    
    if prev is None:
        if startTime is None:
            startTime = x['Time']
        Flow.append(FlowNo)
        
        prev = x['UFid']
    elif compareUF(x['UFid'],prev):
        if x['tcp_Flags']&1:
            Flow.append(FlowNo)
            prev = None
            startTime = None
            FlowNo = FlowNo + 1

        elif x['Time']-startTime>=3600:
            FlowNo = FlowNo + 1
            Flow.append(FlowNo)
            prev = None
            startTime = x['Time']
            
        else:
            
            Flow.append(FlowNo)
            prev = x['UFid']

    else:
        FlowNo = FlowNo + 1
        Flow.append(FlowNo)
        prev = x['UFid']
        startTime = x['Time']


print len(gl.SArray(Flow).unique())

Done Sorting
Running 0 Done !
Running 500000 Done !
Running 1000000 Done !
Running 1500000 Done !
Running 2000000 Done !
Running 2500000 Done !
Running 3000000 Done !
Running 3500000 Done !
Running 4000000 Done !
Running 4500000 Done !
Running 5000000 Done !
Running 5500000 Done !
Running 6000000 Done !
Running 6500000 Done !
Running 7000000 Done !
Running 7500000 Done !
Running 8000000 Done !
Running 8500000 Done !
Running 9000000 Done !
504044


In [38]:
SF['Flow'] = gl.SArray(Flow)
temp = SF.groupby('Flow',{
            'Count':gl.aggregate.COUNT()
        })
len(temp[temp['Count']>1])

404434

In [39]:
temp = SF.groupby('Flow',{
        'NumBots' : gl.aggregate.SUM('isBot')
    })
NumBotFlows = len(temp[temp['NumBots']>1])
print NumBotFlows, NumBotFlows*1.0/len(SF['Flow'].unique()) 

238501 0.473174960916


In [40]:
len(Flow)

9360525

In [41]:
len(SF)

9360525

In [42]:
SF['FlowNo.'] = gl.SArray(Flow)

In [43]:
##Code for checking authenticity of flow logic
#STD[(STD['Source']=='0.0.0.0')&(STD['Destination']=='255.255.255.255')&(STD['Source Port']=='68')&(STD['Destination Port']=='67')].sort('Time')

In [44]:
## Code to check if in any flows there are some No.s which are in decreasing order (Indicative or Decrepancies)
## UPDATE: No. does not indicate same relation in time, so Data collected is right !
"""count = 0
for li in Flow:
    for i in range(1,len(li)):
        if li[i]<li[i-1]:
            #print li
            count = count+1
            break;
print count"""

'count = 0\nfor li in Flow:\n    for i in range(1,len(li)):\n        if li[i]<li[i-1]:\n            #print li\n            count = count+1\n            break;\nprint count'

In [45]:
import pickle
pickle.dump(Flow,open('Flow.pkl','w'))

In [46]:
SF.save('ISCX_Botnet-Testing_Ports_Only_Sorted_Flow_BD.csv')

In [47]:
def tfn(x):
    if 'udp' in x.split(':'):
        return 1
    return 0
SF['hasUDP'] = SF['Protocols in frame'].apply(lambda x:tfn(x))

In [48]:
SF.head


<bound method SFrame.head of Columns:
	No.	int
	Time	float
	Source	str
	Destination	str
	Protocol	str
	Length	int
	Info	str
	Destination Port	str
	Source Port	str
	Protocols in frame	str
	Differentiated Services Field	str
	IP_Flags	str
	udp_Length	str
	TCP Segment Len	str
	Sequence number	str
	tcp_Flags	int
	Answer RRs	str
	Source GeoIP Country	str
	Source GeoIP Latitude	str
	Source GeoIP Longitude	str
	Destination GeoIP Latitude	str
	Destination GeoIP Longitude	str
	Destination GeoIP Country	str
	Time to live	str
	Next sequence number	str
	isBot	int
	UFid	str
	Forward	int
	Flow	int
	FlowNo.	int
	hasUDP	int

Rows: 9360525

Data:
+--------+---------------+---------------+-----------------+----------+--------+
|  No.   |      Time     |     Source    |   Destination   | Protocol | Length |
+--------+---------------+---------------+-----------------+----------+--------+
| 819351 | 453240.146507 | 147.32.84.160 |   1.22.16.143   |   TCP    |   62   |
| 819352 | 453240.146514 | 147.32.84.16

In [50]:
## Ratio of incoming to outgoing packets
temp = SF.groupby('FlowNo.',{
        'NumForward' : gl.aggregate.SUM('Forward'),
        'Total' : gl.aggregate.COUNT()
    })
temp['IOPR']= temp.apply(lambda x: ((x['Total']-x['NumForward'])*1.0)/x['NumForward'] if x['NumForward'] !=0 else (-1) )
temp = temp['FlowNo.','IOPR']

In [57]:
len(temp[temp['IOPR']!=-1])


348937

In [ ]:
SF = SF.join(temp,on='FlowNo.')
SF.head

In [59]:
SF['IOPR']

dtype: float
Rows: 9360525
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ... ]

In [60]:
## First Packet Length
FlowFeatures = ['Source','Destination','Source Port','Destination Port','Protocol']
FPL = SF.groupby(['FlowNo.'],{
        'Time':gl.aggregate.MIN('Time')
    })
print len(FPL)
FPL = FPL.join(SF,on =['FlowNo.','Time'])[['FlowNo.','Length']].unique()
FPL = FPL.groupby(['FlowNo.'],{
        'FPL':gl.aggregate.AVG('Length')
    })
print len(FPL)


# In[33]:

SF = SF.join(FPL, on ='FlowNo.')
del(FPL)


# ## 18/10/2016

# In[35]:

## Number of packets per flow
temp = SF.groupby(['FlowNo.'],{
        'NumPackets':gl.aggregate.COUNT()
    })
print temp.head(3)
SF = SF.join(temp, on ='FlowNo.')
del(temp)


# In[ ]:

## Number of bytes exchanged
temp = SF.groupby(['FlowNo.'],{
        'BytesEx':gl.aggregate.SUM('Length')
    })
SF = SF.join(temp, on ='FlowNo.')
del(temp)


# In[38]:

## Standard deviation of packet length
temp = SF.groupby(['FlowNo.'],{
        'StdDevLen':gl.aggregate.STDV('Length')
    })
SF = SF.join(temp, on ='FlowNo.')
del(temp)


# In[40]:

## Same length packet ratio
temp2 = SF.groupby(['FlowNo.'],{
        'SameLenPktRatio':gl.aggregate.COUNT_DISTINCT('Length')
    })
##temp from number of packets computation
temp = SF.groupby(['FlowNo.'],{
        'NumPackets':gl.aggregate.COUNT()
    })
temp = temp.join(temp2,on='FlowNo.')
temp['SameLenPktRatio'] = temp['SameLenPktRatio']*1.0/temp['NumPackets']
temp2 = None
temp = temp[['FlowNo.','SameLenPktRatio']]
SF = SF.join(temp, on ='FlowNo.')


# In[41]:

## Duration of flow
timeF = SF.groupby(['FlowNo.'],{
        'startTime':gl.aggregate.MIN('Time'),
        'endTime':gl.aggregate.MAX('Time')
    })
timeF['Duration'] = timeF['endTime'] - timeF['startTime']
timeF = timeF[['FlowNo.','Duration']]
SF = SF.join(timeF, on ='FlowNo.')


# In[45]:

sorted(SF.column_names())


504044
504044
+---------+------------+
| FlowNo. | NumPackets |
+---------+------------+
|  211023 |     1      |
|  442699 |     2      |
|  79732  |     4      |
+---------+------------+
[3 rows x 2 columns]



['Answer RRs',
 'BytesEx',
 'Destination',
 'Destination GeoIP Country',
 'Destination GeoIP Latitude',
 'Destination GeoIP Longitude',
 'Destination Port',
 'Differentiated Services Field',
 'Duration',
 'FPL',
 'Flow',
 'FlowNo.',
 'Forward',
 'IOPR',
 'IP_Flags',
 'Info',
 'Length',
 'Next sequence number',
 'No.',
 'NumPackets',
 'Protocol',
 'Protocols in frame',
 'SameLenPktRatio',
 'Sequence number',
 'Source',
 'Source GeoIP Country',
 'Source GeoIP Latitude',
 'Source GeoIP Longitude',
 'Source Port',
 'StdDevLen',
 'TCP Segment Len',
 'Time',
 'Time to live',
 'UFid',
 'hasUDP',
 'isBot',
 'tcp_Flags',
 'udp_Length']

In [61]:


# In[50]:

features = ['Answer RRs',
 'BytesEx',
 'Destination',
 'Destination Port',
 'Differentiated Services Field',
 'Duration',
 'FPL',
 'IP_Flags',
 'Info',
 'Length',
 'Next sequence number',
 'No.',
 'NumPackets',
 'Protocol',
 'Protocols in frame',
 'SameLenPktRatio',
 'Sequence number',
 'Source',
 'Source Port',
 'StdDevLen',
 'TCP Segment Len',
 'Time',
 'Time to live',
 'isBot',
 'tcp_Flags',
 'FlowNo.',
 'udp_Length',
 'IOPR']
SF = SF[features]


# In[52]:

## Average packets per second
temp =  SF.groupby(['FlowNo.'],{
        'NumPackets':gl.aggregate.COUNT()
    })
temp = temp.join(timeF,on=['FlowNo.'])
temp['AvgPktPerSec'] = temp.apply(lambda x:0.0 if x['Duration'] == 0.0 else x['NumPackets']*1.0/x['Duration'])
temp = temp[['FlowNo.','AvgPktPerSec']]
SF = SF.join(temp, on ='FlowNo.')


# In[53]:

##Average Bits Per Second
temp = SF.groupby(['FlowNo.'],{
        'BytesEx':gl.aggregate.SUM('Length')
    })
temp = temp.join(timeF,on=['FlowNo.'])
temp['BitsPerSec'] = temp.apply(lambda x:0.0 if x['Duration'] == 0.0 else x['BytesEx']*8.0/x['Duration'])
temp = temp[['FlowNo.','BitsPerSec']]
SF = SF.join(temp, on ='FlowNo.')


# In[55]:

## Average Packet Lentgth
temp = SF.groupby(['FlowNo.'],{
        'APL':gl.aggregate.AVG('Length')
    })
SF = SF.join(temp, on ='FlowNo.')


# In[ ]:

## Number of Reconnects, sort FlowNo, SeqNo


# In[56]:

def tfn(x):
    if 'udp' in x.split(':') or 'tcp' in x.split(':'):
        return 1
    return 0
temp = list(SF['Protocols in frame'].apply(lambda x:tfn(x)))


# In[57]:

len(temp)


# In[58]:

sum(temp)


# In[60]:



# In[62]:

print len(SF[SF['Protocols in frame']=='eth:ethertype:ip:icmp:ip:tcp:http:urlencoded-form']),len(SF[SF['Protocols in frame']=='eth:ethertype:ip:icmp:ip:tcp']),len(SF[SF['Protocols in frame']=='eth:ethertype:ip:icmp:ip:tcp:http:data'])


# In[64]:

## Inter arrival time
SF['IAT'] = 0
SF = SF.sort(['FlowNo.','Time'])
prev = None
prevT = None
li = []
for x in SF:
    if prev is None or x['FlowNo.']!= prev:
        li.append(0)
    else:
        li.append(x['Time']-prevT)        
    prev = x['FlowNo.']
    prevT = x['Time']
SF['IAT'] = gl.SArray(li)


# In[65]:

len(SF)


# In[66]:

print len(SF[SF['Protocols in frame']=='eth:ipx'])


# In[67]:

SF.save('Bidirectional_Train_Bot_features_till_IAT.csv')


# # Is Null feature

# ### Number of TCP Null packets

# In[68]:


# In[73]:

## Null Packets handling
def checkNull(x):
    if(x['TCP Segment Len']=='0' or x['udp_Length']==8 ):
        return 1
    elif('ipx' in x['Protocols in frame'].split(':')):
        l = x['Length'] - 30
        if('eth' in x['Protocols in frame'].split(':')):
            l = l - 14
        if('ethtype' in x['Protocols in frame'].split(':')):
            l = l - 2
        if('llc' in x['Protocols in frame'].split(':')):
            l = l - 8
        if(l==0 or l==-1):
            return 1
    return 0


# In[74]:

SF['isNull'] = SF.apply(lambda x:checkNull(x))


# In[75]:



# In[76]:

NPEx = SF.groupby(['FlowNo.'],{
        'NPEx':gl.aggregate.SUM('isNull')
    })
SF = SF.join(NPEx, on ='FlowNo.')


# ### Number of Reconnects - considering only TCP reconnects, using sequence number

# In[ ]:



# In[79]:

recon = SF[SF['Sequence number']!=''].groupby(['FlowNo.'],{
        'total_seq_no.' : gl.aggregate.COUNT('Sequence number'),
        'distinct_seq_no.' : gl.aggregate.COUNT_DISTINCT('Sequence number')
    })
recon['reconnects'] = recon['total_seq_no.'] - recon['distinct_seq_no.']
recon.head()
recon = recon[['FlowNo.','reconnects']]
SF = SF.join(recon,on='FlowNo.',how='left')
len(SF)


# In[81]:

#To identify records where reconnect check was not applied like UDP etc
SF.fillna('reconnects',-1)


# In[ ]:

#SF[SF['FlowNo.']==79732]['Info']


# A lot of these reconnects can be simple retransmissions - due to out of order/timeout etcb

# ### 22-10-2016

# In[83]:



# In[15]:

print min(SF['Time']),max(SF['Time'])


# # 28/10/16

# Bidirectional Flowscombining all features and labels

# In[84]:


# In[85]:

SF.save('Bidirectional_Train_Botnet_all_features.csv')


# In[ ]:

#sf_valid_train, sf_test = SF.random_split(.8, seed=5)
#sf_valid, sf_train = sf_valid_train.random_split(.2, seed=5)
#X_train, y_train = sf_train.drop('isBot'), sf_train['isBot']
#X_valid, y_valid = sf_valid_train
#X_train_valid, y_train_valid = X[:3475000], y[:3475000]
#X_test, y_test = X[3475000:], y[3475000:]



# In[89]:

features = SF.column_names()

0 5398 0
0
-121117184.726 780231.692992


In [63]:
SF2 = gl.SFrame.read_csv('ISCX_Botnet-Testing.csv',verbose=False)
print "Done reading"
SF2.head
SF2 = SF2[(SF2['Source Port']!='')&(SF2['Destination Port']!='')]
SF2['tcp_Flags'] = SF2['tcp_Flags'].apply(lambda x:int(x,16) if x!='' else 0)
iplist = [
    "147.32.84.180",
    "147.32.84.170",
    "147.32.84.150",
    "147.32.84.140",
    "147.32.84.130",
    "147.32.84.160",
    "10.0.2.15",
    "192.168.106.141",
    "192.168.106.131",
    "172.16.253.130",
    "172.16.253.131",
    "172.16.253.129",
    "172.16.253.240",
    "74.78.117.238",
    "158.65.110.24",
    "192.168.3.35",
    "192.168.3.25",
    "192.168.3.65",
    "172.29.0.116",
    "172.29.0.109",
    "172.16.253.132",
    "192.168.248.165",
    "10.37.130.4",
    "131.202.243.84", 
    "198.164.30.2", "192.168.5.122", 
    "192.168.5.122", 
    "192.168.5.122", "192.168.5.122", "192.168.5.122", 
    "192.168.2.110", 
    "192.168.4.120", "192.168.1.103",
    "192.168.2.113", "192.168.4.118" , "192.168.2.109", "192.168.2.105", "192.168.5.122"]

MasterBot = [ ("192.168.2.112", "131.202.243.84"), ("192.168.5.122", "198.164.30.2"), ("192.168.2.110", "192.168.5.122"),( "192.168.4.118", "192.168.5.122"), ("192.168.2.113", "192.168.5.122"), ("192.168.1.103", "192.168.5.122"), ("192.168.4.120", "192.168.5.122"), ("192.168.2.112", "192.168.2.110"), ("192.168.2.112", "192.168.4.120"), ("192.168.2.112", "192.168.1.103"), ("192.168.2.112", "192.168.2.113"), ("192.168.2.112", "192.168.4.118"), ("192.168.2.112", "192.168.2.109"), ("192.168.2.112", "192.168.2.105"), ("192.168.1.105", "192.168.5.122")]

iplist = gl.SArray(iplist)
iplist = iplist.unique() 
iplist = list(iplist)

def fcheckIP(x):
    if (x['Source'] in iplist) or (x['Destination'] in iplist):
        return 1
    else:
        if ((x['Source'],x['Destination'])  in MasterBot ) or ((x['Destination'],x['Source'])  in MasterBot ) :
            return 1
        else:
            return 0
        

SF2['isBot'] = SF2[['Source','Destination']].apply(lambda x: fcheckIP(x))

def flow_id(x):
    if x['Source']>x['Destination']:
        return x['Source']+'-'+x['Destination']+'-'+str(x['Source Port'])+'-'+str(x['Destination Port'])+'-'+x['Protocol']
    else:
        return x['Destination']+'-'+x['Source']+'-'+str(x['Destination Port'])+'-'+str(x['Source Port'])+'-'+x['Protocol']
SF2['UFid'] = SF2.apply(lambda x:flow_id(x))


#For identifying IOPR
SF2['Forward'] = SF2.apply(lambda x: 1 if x['Source']>x['Destination'] else 0 )


## function for comparing two different flows based on columns
def compareUF(x,y):
    if x!=y:
        return False
    return True


##Code for logic of Bidirectional flow identification
import pickle 

FlowNo = 0 ##Unique Flow Number for each flow
prev = None
Flow = []     ##Stores all flows in form of list of dictionary 
#cFlow = []    ##Store the current flow (all details)
count = 0
fc = 0
startTime = None   ##Start Time of each flow to implement timeout
SF2 = SF2.sort(['UFid','Time'])
print 'Done Sorting'
for x in SF2:
    if count%500000 == 0:
        print 'Running '+str(count)+' Done !'
        
    count = count+1
    
    if prev is None:
        if startTime is None:
            startTime = x['Time']
        Flow.append(FlowNo)
        
        prev = x['UFid']
    elif compareUF(x['UFid'],prev):
        if x['tcp_Flags']&1:
            Flow.append(FlowNo)
            prev = None
            startTime = None
            FlowNo = FlowNo + 1

        elif x['Time']-startTime>=3600:
            FlowNo = FlowNo + 1
            Flow.append(FlowNo)
            prev = None
            startTime = x['Time']
            
        else:
            
            Flow.append(FlowNo)
            prev = x['UFid']

    else:
        FlowNo = FlowNo + 1
        Flow.append(FlowNo)
        prev = x['UFid']
        startTime = x['Time']


print len(gl.SArray(Flow).unique())


SF2['Flow'] = gl.SArray(Flow)
temp = SF2.groupby('Flow',{
            'Count':gl.aggregate.COUNT()
        })
len(temp[temp['Count']>1])


temp = SF2.groupby('Flow',{
        'NumBots' : gl.aggregate.SUM('isBot')
    })
NumBotFlows = len(temp[temp['NumBots']>1])
print NumBotFlows, NumBotFlows*1.0/len(SF2['Flow'].unique()) 

SF2['FlowNo.'] = gl.SArray(Flow)

import pickle
pickle.dump(Flow,open('Flow.pkl','w'))

SF2.save('ISCX_Botnet-Testing_Ports_Only_Sorted_Flow_BD.csv')


def tfn(x):
    if 'udp' in x.split(':'):
        return 1
    return 0
SF2['hasUDP'] = SF2['Protocols in frame'].apply(lambda x:tfn(x))


## Ratio of incoming to outgoing packets
temp = SF2.groupby('FlowNo.',{
        'NumForward' : gl.aggregate.SUM('Forward'),
        'Total' : gl.aggregate.COUNT()
    })
temp['IOPR']= temp.apply(lambda x: ((x['Total']-x['NumForward'])*1.0)/x['NumForward'] if x['NumForward'] !=0 else (-1) )
temp = temp['FlowNo.','IOPR']


SF2 = SF2.join(temp,on='FlowNo.')
SF2.head

## First Packet Length
FlowFeatures = ['Source','Destination','Source Port','Destination Port','Protocol']
FPL = SF2.groupby(['FlowNo.'],{
        'Time':gl.aggregate.MIN('Time')
    })
print len(FPL)
FPL = FPL.join(SF2,on =['FlowNo.','Time'])[['FlowNo.','Length']].unique()
FPL = FPL.groupby(['FlowNo.'],{
        'FPL':gl.aggregate.AVG('Length')
    })
print len(FPL)


# In[33]:

SF2 = SF2.join(FPL, on ='FlowNo.')
del(FPL)


# ## 18/10/2016

# In[35]:

## Number of packets per flow
temp = SF2.groupby(['FlowNo.'],{
        'NumPackets':gl.aggregate.COUNT()
    })
print temp.head(3)
SF2 = SF2.join(temp, on ='FlowNo.')
del(temp)


# In[ ]:

## Number of bytes exchanged
temp = SF2.groupby(['FlowNo.'],{
        'BytesEx':gl.aggregate.SUM('Length')
    })
SF2 = SF2.join(temp, on ='FlowNo.')
del(temp)


# In[38]:

## Standard deviation of packet length
temp = SF2.groupby(['FlowNo.'],{
        'StdDevLen':gl.aggregate.STDV('Length')
    })
SF2 = SF2.join(temp, on ='FlowNo.')
del(temp)


# In[40]:

## Same length packet ratio
temp2 = SF2.groupby(['FlowNo.'],{
        'SameLenPktRatio':gl.aggregate.COUNT_DISTINCT('Length')
    })
##temp from number of packets computation
temp = SF2.groupby(['FlowNo.'],{
        'NumPackets':gl.aggregate.COUNT()
    })
temp = temp.join(temp2,on='FlowNo.')
temp['SameLenPktRatio'] = temp['SameLenPktRatio']*1.0/temp['NumPackets']
temp2 = None
temp = temp[['FlowNo.','SameLenPktRatio']]
SF2 = SF2.join(temp, on ='FlowNo.')


# In[41]:

## Duration of flow
timeF = SF2.groupby(['FlowNo.'],{
        'startTime':gl.aggregate.MIN('Time'),
        'endTime':gl.aggregate.MAX('Time')
    })
timeF['Duration'] = timeF['endTime'] - timeF['startTime']
timeF = timeF[['FlowNo.','Duration']]
SF2 = SF2.join(timeF, on ='FlowNo.')


# In[45]:

sorted(SF2.column_names())





# In[50]:

features = ['Answer RRs',
 'BytesEx',
 'Destination',
 'Destination Port',
 'Differentiated Services Field',
 'Duration',
 'FPL',
 'IP_Flags',
 'Info',
 'Length',
 'Next sequence number',
 'No.',
 'NumPackets',
 'Protocol',
 'Protocols in frame',
 'SameLenPktRatio',
 'Sequence number',
 'Source',
 'Source Port',
 'StdDevLen',
 'TCP Segment Len',
 'Time',
 'Time to live',
 'isBot',
 'tcp_Flags',
 'FlowNo.',
 'udp_Length',
 'IOPR']
SF2 = SF2[features]


# In[52]:

## Average packets per second
temp =  SF2.groupby(['FlowNo.'],{
        'NumPackets':gl.aggregate.COUNT()
    })
temp = temp.join(timeF,on=['FlowNo.'])
temp['AvgPktPerSec'] = temp.apply(lambda x:0.0 if x['Duration'] == 0.0 else x['NumPackets']*1.0/x['Duration'])
temp = temp[['FlowNo.','AvgPktPerSec']]
SF2 = SF2.join(temp, on ='FlowNo.')


# In[53]:

##Average Bits Per Second
temp = SF2.groupby(['FlowNo.'],{
        'BytesEx':gl.aggregate.SUM('Length')
    })
temp = temp.join(timeF,on=['FlowNo.'])
temp['BitsPerSec'] = temp.apply(lambda x:0.0 if x['Duration'] == 0.0 else x['BytesEx']*8.0/x['Duration'])
temp = temp[['FlowNo.','BitsPerSec']]
SF2 = SF2.join(temp, on ='FlowNo.')


# In[55]:

## Average Packet Lentgth
temp = SF2.groupby(['FlowNo.'],{
        'APL':gl.aggregate.AVG('Length')
    })
SF2 = SF2.join(temp, on ='FlowNo.')


# In[ ]:

## Number of Reconnects, sort FlowNo, SeqNo


# In[56]:

def tfn(x):
    if 'udp' in x.split(':') or 'tcp' in x.split(':'):
        return 1
    return 0
temp = list(SF2['Protocols in frame'].apply(lambda x:tfn(x)))


# In[57]:

len(temp)


# In[58]:

sum(temp)


# In[60]:



# In[62]:

print len(SF2[SF2['Protocols in frame']=='eth:ethertype:ip:icmp:ip:tcp:http:urlencoded-form']),len(SF2[SF2['Protocols in frame']=='eth:ethertype:ip:icmp:ip:tcp']),len(SF2[SF2['Protocols in frame']=='eth:ethertype:ip:icmp:ip:tcp:http:data'])


# In[64]:

## Inter arrival time
SF2['IAT'] = 0
SF2 = SF2.sort(['FlowNo.','Time'])
prev = None
prevT = None
li = []
for x in SF2:
    if prev is None or x['FlowNo.']!= prev:
        li.append(0)
    else:
        li.append(x['Time']-prevT)        
    prev = x['FlowNo.']
    prevT = x['Time']
SF2['IAT'] = gl.SArray(li)


# In[65]:

len(SF2)


# In[66]:

print len(SF2[SF2['Protocols in frame']=='eth:ipx'])


# In[67]:

SF2.save('Bidirectional_Test_Bot_features_till_IAT.csv')


# # Is Null feature

# ### Number of TCP Null packets

# In[68]:


# In[73]:

## Null Packets handling
def checkNull(x):
    if(x['TCP Segment Len']=='0' or x['udp_Length']==8 ):
        return 1
    elif('ipx' in x['Protocols in frame'].split(':')):
        l = x['Length'] - 30
        if('eth' in x['Protocols in frame'].split(':')):
            l = l - 14
        if('ethtype' in x['Protocols in frame'].split(':')):
            l = l - 2
        if('llc' in x['Protocols in frame'].split(':')):
            l = l - 8
        if(l==0 or l==-1):
            return 1
    return 0


# In[74]:

SF2['isNull'] = SF2.apply(lambda x:checkNull(x))


# In[75]:



# In[76]:

NPEx = SF2.groupby(['FlowNo.'],{
        'NPEx':gl.aggregate.SUM('isNull')
    })
SF2 = SF2.join(NPEx, on ='FlowNo.')


# ### Number of Reconnects - considering only TCP reconnects, using sequence number

# In[ ]:



# In[79]:

recon = SF2[SF2['Sequence number']!=''].groupby(['FlowNo.'],{
        'total_seq_no.' : gl.aggregate.COUNT('Sequence number'),
        'distinct_seq_no.' : gl.aggregate.COUNT_DISTINCT('Sequence number')
    })
recon['reconnects'] = recon['total_seq_no.'] - recon['distinct_seq_no.']
recon.head()
recon = recon[['FlowNo.','reconnects']]
SF2 = SF2.join(recon,on='FlowNo.',how='left')
len(SF2)


# In[81]:

#To identify records where reconnect check was not applied like UDP etc
SF2.fillna('reconnects',-1)


# In[ ]:

#SF2[SF2['FlowNo.']==79732]['Info']


# A lot of these reconnects can be simple retransmissions - due to out of order/timeout etcb

# ### 22-10-2016

# In[83]:



# In[15]:

print min(SF2['Time']),max(SF2['Time'])


# # 28/10/16

# Bidirectional Flowscombining all features and labels

# In[84]:


# In[85]:

SF2.save('Bidirectional_Test_Botnet_all_features.csv')


# In[ ]:

#SF2_valid_Test, SF2_test = SF2.random_split(.8, seed=5)
#SF2_valid, SF2_Test = SF2_valid_Test.random_split(.2, seed=5)
#X_Test, y_Test = SF2_Test.drop('isBot'), SF2_Test['isBot']
#X_valid, y_valid = SF2_valid_Test
#X_Test_valid, y_Test_valid = X[:3475000], y[:3475000]
#X_test, y_test = X[3475000:], y[3475000:]



# In[89]:

features = SF2.column_names()

Done reading
Done Sorting
Running 0 Done !
Running 500000 Done !
Running 1000000 Done !
Running 1500000 Done !
Running 2000000 Done !
Running 2500000 Done !
Running 3000000 Done !
Running 3500000 Done !
Running 4000000 Done !
Running 4500000 Done !
355434
216964 0.610419937316
355434
355434
+---------+------------+
| FlowNo. | NumPackets |
+---------+------------+
|  211023 |     6      |
|  79732  |     1      |
|  333842 |     7      |
+---------+------------+
[3 rows x 2 columns]

1 234 1
5637
6.985427 1359941622.99


In [77]:
features =SF2.column_names()
features

['Answer RRs',
 'BytesEx',
 'Destination',
 'Destination Port',
 'Differentiated Services Field',
 'Duration',
 'FPL',
 'IP_Flags',
 'Info',
 'Length',
 'Next sequence number',
 'No.',
 'NumPackets',
 'Protocol',
 'Protocols in frame',
 'SameLenPktRatio',
 'Sequence number',
 'Source',
 'Source Port',
 'StdDevLen',
 'TCP Segment Len',
 'Time',
 'Time to live',
 'isBot',
 'tcp_Flags',
 'FlowNo.',
 'udp_Length',
 'IOPR',
 'AvgPktPerSec',
 'BitsPerSec',
 'APL',
 'IAT',
 'isNull',
 'NPEx',
 'reconnects']

In [72]:
features

NameError: name 'features' is not defined

In [75]:
myfeatures

['BytesEx',
 'Destination',
 'Destination Port',
 'Differentiated Services Field',
 'Duration',
 'FPL',
 'Info',
 'Length',
 'NumPackets',
 'Protocol',
 'SameLenPktRatio',
 'Source',
 'Source Port',
 'StdDevLen',
 'TCP Segment Len',
 'Time to live',
 'FlowNo.',
 'udp_Length',
 'IOPR',
 'AvgPktPerSec',
 'BitsPerSec',
 'APL',
 'IAT',
 'isNull',
 'NPEx',
 'reconnects']

15-11-2016

In [2]:
SF = gl.SFrame.read_csv('Bidirectional_Train_Botnet_all_features.csv',verbose=False)

In [3]:
len(SF)

9360525

In [4]:
SF['Forward'] = SF.apply(lambda x: 1 if x['Source']>x['Destination'] else 0 )
temp = SF.groupby('FlowNo.',{
        'NumForward' : gl.aggregate.SUM('Forward'),
        
    })

SF = SF.join(temp,on='FlowNo.')

In [5]:
SF.head

<bound method SFrame.head of Columns:
	Answer RRs	str
	BytesEx	int
	Destination	str
	Destination Port	int
	Differentiated Services Field	str
	Duration	float
	FPL	int
	IP_Flags	str
	Info	str
	Length	int
	Next sequence number	str
	No.	int
	NumPackets	int
	Protocol	str
	Protocols in frame	str
	SameLenPktRatio	float
	Sequence number	str
	Source	str
	Source Port	int
	StdDevLen	float
	TCP Segment Len	int
	Time	int
	Time to live	str
	isBot	int
	tcp_Flags	int
	FlowNo.	int
	udp_Length	str
	IOPR	float
	AvgPktPerSec	float
	BitsPerSec	float
	APL	float
	IAT	float
	isNull	int
	NPEx	int
	reconnects	int
	Forward	int
	NumForward	int

Rows: 9360525

Data:
+------------+---------+-----------------+------------------+-------------------------------+
| Answer RRs | BytesEx |   Destination   | Destination Port | Differentiated Services Field |
+------------+---------+-----------------+------------------+-------------------------------+
|            |   372   |   1.22.16.143   |       3128       |           

In [6]:
Test = gl.SFrame.read_csv('Bidirectional_Test_Botnet_all_features.csv',verbose=False)
Test.head

<bound method SFrame.head of Columns:
	Answer RRs	str
	BytesEx	int
	Destination	str
	Destination Port	str
	Differentiated Services Field	str
	Duration	float
	FPL	int
	IP_Flags	str
	Info	str
	Length	int
	Next sequence number	str
	No.	int
	NumPackets	int
	Protocol	str
	Protocols in frame	str
	SameLenPktRatio	float
	Sequence number	str
	Source	str
	Source Port	str
	StdDevLen	float
	TCP Segment Len	str
	Time	float
	Time to live	str
	isBot	int
	tcp_Flags	int
	FlowNo.	int
	udp_Length	str
	IOPR	int
	AvgPktPerSec	float
	BitsPerSec	float
	APL	float
	IAT	float
	isNull	int
	NPEx	int
	reconnects	str

Rows: 4676319

Data:
+------------+---------+-----------------------+------------------+-------------------------------+
| Answer RRs | BytesEx |      Destination      | Destination Port | Differentiated Services Field |
+------------+---------+-----------------------+------------------+-------------------------------+
|            |  27511  | 00000000.ffffffffffff |      0x0452      |                

In [7]:
for col in ['Source Port','Destination Port']:
    SF[col] = SF[col].apply(lambda x: str(x))
    Test[col] = Test[col].apply(lambda x: str(x))

In [8]:
Test['Forward'] = Test.apply(lambda x: 1 if x['Source']>x['Destination'] else 0 )
temp = Test.groupby('FlowNo.',{
        'NumForward' : gl.aggregate.SUM('Forward'),
        
    })

Test= Test.join(temp,on='FlowNo.')

In [9]:
Test = Test.groupby('FlowNo.',{
        'Answer RRs': gl.aggregate.SELECT_ONE('Answer RRs'),
        'BytesEx' : gl.aggregate.SELECT_ONE('BytesEx'),
        'Destination' : gl.aggregate.SELECT_ONE('Destination'),
        'Destination Port' : gl.aggregate.SELECT_ONE('Destination Port'),
        'Differentiated Services Field' : gl.aggregate.SELECT_ONE('Differentiated Services Field'),
        'Duration' : gl.aggregate.SELECT_ONE('Duration'),
        'FPL' : gl.aggregate.SELECT_ONE('FPL'),
        'IP_Flags' : gl.aggregate.SELECT_ONE('IP_Flags'),
        'Info' : gl.aggregate.SELECT_ONE('Info'),
        'Length' : gl.aggregate.SELECT_ONE('Length'),
        'Next sequence number' : gl.aggregate.SELECT_ONE('Next sequence number'),
        'No.' : gl.aggregate.SELECT_ONE('No.'),
        'NumPackets' : gl.aggregate.SELECT_ONE('NumPackets'),
        'Protocol' : gl.aggregate.SELECT_ONE('Protocol'),
        'Protocols in frame' : gl.aggregate.SELECT_ONE('Protocols in frame'),
        'SameLenPktRatio' : gl.aggregate.SELECT_ONE('SameLenPktRatio'),
        'Sequence number' : gl.aggregate.SELECT_ONE('Sequence number'),
        'Source' : gl.aggregate.SELECT_ONE('Source'),
        'Source Port' : gl.aggregate.SELECT_ONE('Source Port'),
        'StdDevLen' : gl.aggregate.SELECT_ONE('StdDevLen'),
        'IAT' : gl.aggregate.SELECT_ONE('IAT'),
        'isNull' : gl.aggregate.SELECT_ONE('isNull'),
        'NPEx' : gl.aggregate.SELECT_ONE('NPEx'),
        'reconnects' : gl.aggregate.SELECT_ONE('reconnects'),
        'APL' : gl.aggregate.SELECT_ONE('APL'),
        'BitsPerSec' : gl.aggregate.SELECT_ONE('BitsPerSec'),
        'AvgPktPerSec' : gl.aggregate.SELECT_ONE('AvgPktPerSec'),
        'udp_Length' : gl.aggregate.SELECT_ONE('udp_Length'),
        'tcp_Flags' : gl.aggregate.SELECT_ONE('tcp_Flags'),
        'isBot' : gl.aggregate.SELECT_ONE('isBot'),
        'Time to live' : gl.aggregate.SELECT_ONE('Time to live'),
        'Time' : gl.aggregate.SELECT_ONE('Time'),
        'TCP Segment Len' : gl.aggregate.SELECT_ONE('TCP Segment Len'),
        'IOPR' : gl.aggregate.SELECT_ONE('IOPR'),
        'NumForward' : gl.aggregate.SELECT_ONE('NumForward')
    })



In [10]:
SF = SF.groupby('FlowNo.',{
        'Answer RRs': gl.aggregate.SELECT_ONE('Answer RRs'),
        'BytesEx' : gl.aggregate.SELECT_ONE('BytesEx'),
        'Destination' : gl.aggregate.SELECT_ONE('Destination'),
        'Destination Port' : gl.aggregate.SELECT_ONE('Destination Port'),
        'Differentiated Services Field' : gl.aggregate.SELECT_ONE('Differentiated Services Field'),
        'Duration' : gl.aggregate.SELECT_ONE('Duration'),
        'FPL' : gl.aggregate.SELECT_ONE('FPL'),
        'IP_Flags' : gl.aggregate.SELECT_ONE('IP_Flags'),
        'Info' : gl.aggregate.SELECT_ONE('Info'),
        'Length' : gl.aggregate.SELECT_ONE('Length'),
        'Next sequence number' : gl.aggregate.SELECT_ONE('Next sequence number'),
        'No.' : gl.aggregate.SELECT_ONE('No.'),
        'NumPackets' : gl.aggregate.SELECT_ONE('NumPackets'),
        'Protocol' : gl.aggregate.SELECT_ONE('Protocol'),
        'Protocols in frame' : gl.aggregate.SELECT_ONE('Protocols in frame'),
        'SameLenPktRatio' : gl.aggregate.SELECT_ONE('SameLenPktRatio'),
        'Sequence number' : gl.aggregate.SELECT_ONE('Sequence number'),
        'Source' : gl.aggregate.SELECT_ONE('Source'),
        'Source Port' : gl.aggregate.SELECT_ONE('Source Port'),
        'StdDevLen' : gl.aggregate.SELECT_ONE('StdDevLen'),
        'IAT' : gl.aggregate.SELECT_ONE('IAT'),
        'isNull' : gl.aggregate.SELECT_ONE('isNull'),
        'NPEx' : gl.aggregate.SELECT_ONE('NPEx'),
        'reconnects' : gl.aggregate.SELECT_ONE('reconnects'),
        'APL' : gl.aggregate.SELECT_ONE('APL'),
        'BitsPerSec' : gl.aggregate.SELECT_ONE('BitsPerSec'),
        'AvgPktPerSec' : gl.aggregate.SELECT_ONE('AvgPktPerSec'),
        'udp_Length' : gl.aggregate.SELECT_ONE('udp_Length'),
        'tcp_Flags' : gl.aggregate.SELECT_ONE('tcp_Flags'),
        'isBot' : gl.aggregate.SELECT_ONE('isBot'),
        'Time to live' : gl.aggregate.SELECT_ONE('Time to live'),
        'Time' : gl.aggregate.SELECT_ONE('Time'),
        'TCP Segment Len' : gl.aggregate.SELECT_ONE('TCP Segment Len'),
        'IOPR' : gl.aggregate.SELECT_ONE('IOPR'),
        'NumForward' : gl.aggregate.SELECT_ONE('NumForward')
    })



In [11]:
iplist = [
    "147.32.84.180",
    "147.32.84.170",
    "147.32.84.150",
    "147.32.84.140",
    "147.32.84.130",
    "147.32.84.160",
    "10.0.2.15",
    "192.168.106.141",
    "192.168.106.131",
    "172.16.253.130",
    "172.16.253.131",
    "172.16.253.129",
    "172.16.253.240",
    "74.78.117.238",
    "158.65.110.24",
    "192.168.3.35",
    "192.168.3.25",
    "192.168.3.65",
    "172.29.0.116",
    "172.29.0.109",
    "172.16.253.132",
    "192.168.248.165",
    "10.37.130.4",
    "131.202.243.84", 
    "198.164.30.2", "192.168.5.122", 
    "192.168.5.122", 
    "192.168.5.122", "192.168.5.122", "192.168.5.122", 
    "192.168.2.110", 
    "192.168.4.120", "192.168.1.103",
    "192.168.2.113", "192.168.4.118" , "192.168.2.109", "192.168.2.105", "192.168.5.122"]

MasterBot = [ ("192.168.2.112", "131.202.243.84"), ("192.168.5.122", "198.164.30.2"), ("192.168.2.110", "192.168.5.122"),( "192.168.4.118", "192.168.5.122"), ("192.168.2.113", "192.168.5.122"), ("192.168.1.103", "192.168.5.122"), ("192.168.4.120", "192.168.5.122"), ("192.168.2.112", "192.168.2.110"), ("192.168.2.112", "192.168.4.120"), ("192.168.2.112", "192.168.1.103"), ("192.168.2.112", "192.168.2.113"), ("192.168.2.112", "192.168.4.118"), ("192.168.2.112", "192.168.2.109"), ("192.168.2.112", "192.168.2.105"), ("192.168.1.105", "192.168.5.122")]

iplist = gl.SArray(iplist)
iplist = iplist.unique() 
iplist = list(iplist)

def fcheckIP(x):
    if ((x['NumForward']==0) and (x['Destination'] in iplist) ) or ((x['NumForward']==x['NumPackets']) and (x['Source'] in iplist) ):
        return 1
    elif ((x['Source'] in iplist) or (x['Destination'] in iplist)) and (x['NumForward']!=0) and (x['NumForward']!=x['NumPackets']):
        return 1
    else:
        if ((x['Source'],x['Destination'])  in MasterBot ) or ((x['Destination'],x['Source'])  in MasterBot ) :
            return 1
        else:
            return 0
        

SF['isBot'] = SF[['Source','Destination','NumForward','NumPackets']].apply(lambda x: fcheckIP(x))

In [12]:
temp = SF.groupby('FlowNo.',{
        'NumBots' : gl.aggregate.SUM('isBot')
    })
NumBotFlows = len(temp[temp['NumBots']>=1])
print NumBotFlows, NumBotFlows*1.0/len(SF['FlowNo.'].unique()) 

185311 0.367648459261


In [13]:
Test['isBot'] = Test[['Source','Destination','NumForward','NumPackets']].apply(lambda x: fcheckIP(x))

temp = Test.groupby('FlowNo.',{
        'NumBots' : gl.aggregate.SUM('isBot')
    })
NumBotFlows = len(temp[temp['NumBots']>=1])
print NumBotFlows, NumBotFlows*1.0/len(Test['FlowNo.'].unique()) 

169686 0.477405087864


In [14]:
len(Test[Test['isBot']==1]),len(Test)

(169686, 355434)

In [15]:
len(SF[SF['isBot']==1]),len(SF)

(185311, 504044)

In [16]:
Test.save('Bidirectional_Botnet_Test_Final_Flow_Based_Features.csv')
SF.save('Bidirectional_Botnet_Training_Final_Flow_Based_Features.csv')

In [16]:


models = []

myfeatures = Test.column_names()
for x in ['isBot','Answer RRs','Sequence number','No.','IP_Flags','Next sequence number','Protocols in frame','Time','tcp_Flags','FlowNo.','udp_Length']:
    #print x
    #if x not in myfeatures:
        #print x 
    myfeatures.remove(x)
    

In [17]:
myfeatures

['Info',
 'Destination',
 'APL',
 'AvgPktPerSec',
 'Destination Port',
 'TCP Segment Len',
 'Differentiated Services Field',
 'IOPR',
 'Source',
 'Length',
 'IAT',
 'reconnects',
 'NumForward',
 'Protocol',
 'BytesEx',
 'isNull',
 'BitsPerSec',
 'NumPackets',
 'Source Port',
 'StdDevLen',
 'SameLenPktRatio',
 'FPL',
 'Duration',
 'Time to live',
 'NPEx']

In [19]:
l=0
for x in iplist:
    l=l+ len(Test[(Test['IOPR']==-1) & (Test['isBot']==1) & (Test['Source']==x)])
print "Number of flows having only destination to source, and source as malicious " + str(l)

Number of flows having only destination to source, and source as malicious 247


In [20]:
l=0
for x in iplist:
    l=l+ len(SF[(SF['IOPR']==-1) & (SF['isBot']==1) & (SF['Source']==x)])
print "Number of flows having only destination to source, and source as malicious " + str(l)

Number of flows having only destination to source, and source as malicious 2159


In [134]:
l=0
for x in iplist:
    l=l+ len(SF[(SF['NumForward']==0) & (SF['isBot']==1) & (SF['Source']==x)])
print "Number of flows having only destination to source, and source as malicious " + str(l)

Number of flows having only destination to source, and source as malicious 73443


In [101]:
len(SF[(SF['IOPR']==0) & (SF['isBot']==1) ])

50535

In [103]:
len(SF[(SF['IOPR']==0) | (SF['IOPR']==-1)])

248083

In [80]:
len(Test[Test['IOPR']==-1]),len(Test)

(92524, 355434)

In [81]:
len(SF[SF['IOPR']==-1]),len(SF)

(155107, 504044)

In [18]:
for x in ['Source','Source Port','Destination','Destination Port','Differentiated Services Field','TCP Segment Len','Length']:
    myfeatures.remove(x)
    


In [22]:
myfeatures1 = ['APL','IOPR','BitsPerSec','Duration']

In [19]:
myfeatures.remove('isNull')
myfeatures.remove('Info')
myfeatures.remove('Time to live')

In [59]:
len(Test[Test['reconnects']==''])

94391

In [20]:
myfeatures.remove('reconnects')

In [21]:
myfeatures.remove('IOPR')

In [22]:
myfeatures

['APL',
 'AvgPktPerSec',
 'IAT',
 'NumForward',
 'Protocol',
 'BytesEx',
 'BitsPerSec',
 'NumPackets',
 'StdDevLen',
 'SameLenPktRatio',
 'FPL',
 'Duration',
 'NPEx']

In [23]:
len(Test[Test['isBot']==0]),len(Test[Test['isBot']==1])

(185748, 169686)

In [24]:
len(SF[SF['isBot']==0]),len(SF[SF['isBot']==1])

(318733, 185311)

In [25]:
SF = SF[SF['NumPackets']>1]
Test = Test[Test['NumPackets']>1]

In [28]:
SF = gl.cross_validation.shuffle(SF)
Folds = gl.cross_validation.KFold(SF,5)
j=0
for train,val in Folds:
    model = gl.boosted_trees_classifier.create(train,features=myfeatures,target='isBot',validation_set=val,verbose = True,max_iterations=100)
    models.append(model)
    print "Done",j
    j=j+1
    
pred = []
for model in models:
    temp = list(model.predict(Test))
    pred.append(temp)
    
final = []
for i in range(len(Test)):
    count = 0
    for j in range(5):
        if pred[j][i] ==1:
            count += 1
    if count>2:
        final.append(1)
    else:
        final.append(0)
        
print gl.toolkits.evaluation.auc(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.accuracy(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.confusion_matrix(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.f1_score(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.log_loss(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.precision(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.recall(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.roc_curve(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.roc_curve(Test['isBot'],gl.SArray(final))


Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 323547

Number of classes           : 2

Number of feature columns   : 13

...

| 90        | 20.287368    | 0.884963          | 0.876864            | 0.246291          | 0.254748            |

| 95        | 21.370096    | 0.886119          | 0.877692            | 0.244133          | 0.253002            |

| 100       | 22.415730    | 0.887185          | 0.878792            | 0.241564          | 0.251105            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

Done 4
0.761380692026
0.821533516399
+--------------+-----------------+--------+
| target_label | predicted_label | count  |
+--------------+-----------------+--------+
|      0       |        1        | 22251  |
|      0       |        0        | 121829 |
|      1       |        1        | 134266 |
|      1       |        0        | 33382  |
+--------------+-----------------+--------+
[4 rows x 3 columns]

0.828380608641
6.16401397108
0.857836528939
0.80088041611
+-----------+----------------+---------------+--------+--------+
| threshold |      fpr       |      tpr      |   p    |   n    |
+-----------+----------------+---------------+--------+--------+
|    0.0    |      1.0       |      1.0      | 167648 | 144080 |
|   1e-05   | 0.154435036091 | 0.80088041611 | 167648 | 144080 |
|   2e-05   | 0.154435036091 | 0.80088041611 | 167648 | 144080 |
|   3e-05   | 0.154435036091 | 0.80088041611 | 167648 | 144080 |
|   4e-05   | 0.154435036091 | 0.80088041611 | 167648 | 144080 |
|   5e-05  

In [29]:
models = []
    
#LOGISTIC REGRESSION
j=1
for train,val in Folds:
    model = gl.logistic_classifier.create(train,features=myfeatures,target='isBot',validation_set=val,verbose = False,max_iterations=100)
    models.append(model)
    print "Done",j
    j=j+1
    
pred = []
for model in models:
    temp = list(model.predict(Test))
    pred.append(temp)
    
final = []
for i in range(len(Test)):
    count = 0
    for j in range(5):
        if pred[j][i] ==1:
            count += 1
    if count>2:
        final.append(1)
    else:
        final.append(0)
        
print gl.toolkits.evaluation.auc(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.accuracy(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.confusion_matrix(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.f1_score(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.log_loss(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.precision(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.recall(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.roc_curve(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.roc_curve(Test['isBot'],gl.SArray(final))


Done 4
Done 5
Done 6
Done 7
Done 8
0.610836556598
0.636901401222
+--------------+-----------------+--------+
| target_label | predicted_label | count  |
+--------------+-----------------+--------+
|      0       |        1        | 24344  |
|      0       |        0        | 119736 |
|      1       |        1        | 78804  |
|      1       |        0        | 88844  |
+--------------+-----------------+--------+
[4 rows x 3 columns]

0.582017459638
12.5409813125
0.763989607166
0.470056308456
+-----------+----------------+----------------+--------+--------+
| threshold |      fpr       |      tpr       |   p    |   n    |
+-----------+----------------+----------------+--------+--------+
|    0.0    |      1.0       |      1.0       | 167648 | 144080 |
|   1e-05   | 0.168961687951 | 0.470056308456 | 167648 | 144080 |
|   2e-05   | 0.168961687951 | 0.470056308456 | 167648 | 144080 |
|   3e-05   | 0.168961687951 | 0.470056308456 | 167648 | 144080 |
|   4e-05   | 0.168961687951 | 0.4700563

In [30]:
models = []
SF = gl.cross_validation.shuffle(SF)
Folds = gl.cross_validation.KFold(SF,5)
j=0
for train,val in Folds:
    model = gl.random_forest_classifier.create(train,features=myfeatures,target='isBot',validation_set=val,verbose = False,max_iterations=100)
    models.append(model)
    print "Done",j
    j=j+1
    
pred = []
for model in models:
    temp = list(model.predict(Test))
    pred.append(temp)
    
final = []
for i in range(len(Test)):
    count = 0
    for j in range(5):
        if pred[j][i] ==1:
            count += 1
    if count>2:
        final.append(1)
    else:
        final.append(0)
        
print gl.toolkits.evaluation.auc(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.accuracy(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.confusion_matrix(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.f1_score(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.log_loss(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.precision(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.recall(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.roc_curve(Test['isBot'],gl.SArray(final))
print gl.toolkits.evaluation.roc_curve(Test['isBot'],gl.SArray(final))

Done 0
Done 1
Done 2
Done 3
Done 4
0.66963854541
0.761840450649
+--------------+-----------------+--------+
| target_label | predicted_label | count  |
+--------------+-----------------+--------+
|      0       |        1        | 34699  |
|      0       |        0        | 109381 |
|      1       |        1        | 128106 |
|      1       |        0        | 39542  |
+--------------+-----------------+--------+
[4 rows x 3 columns]

0.775335675573
8.22573942134
0.786867725193
0.764136762741
+-----------+---------------+----------------+--------+--------+
| threshold |      fpr      |      tpr       |   p    |   n    |
+-----------+---------------+----------------+--------+--------+
|    0.0    |      1.0      |      1.0       | 167648 | 144080 |
|   1e-05   | 0.24083148251 | 0.764136762741 | 167648 | 144080 |
|   2e-05   | 0.24083148251 | 0.764136762741 | 167648 | 144080 |
|   3e-05   | 0.24083148251 | 0.764136762741 | 167648 | 144080 |
|   4e-05   | 0.24083148251 | 0.764136762741 | 1

In [31]:
SF.save('FinalTrain.csv')
Test.save('FinalTest.csv')

In [32]:
SF.column_names()

['FlowNo.',
 'Info',
 'udp_Length',
 'Destination',
 'APL',
 'AvgPktPerSec',
 'Destination Port',
 'Answer RRs',
 'TCP Segment Len',
 'Differentiated Services Field',
 'IOPR',
 'Source',
 'Length',
 'Time',
 'IAT',
 'reconnects',
 'NumForward',
 'tcp_Flags',
 'Protocols in frame',
 'No.',
 'Protocol',
 'BytesEx',
 'isNull',
 'BitsPerSec',
 'isBot',
 'IP_Flags',
 'NumPackets',
 'Source Port',
 'StdDevLen',
 'Next sequence number',
 'SameLenPktRatio',
 'FPL',
 'Duration',
 'Time to live',
 'Sequence number',
 'NPEx']

In [ ]:
SF[SF['AnswerRRs']>1]

In [11]:
len(SF[SF['IOPR']==-1])

582429

In [13]:
iplist = [
    "147.32.84.180",
    "147.32.84.170",
    "147.32.84.150",
    "147.32.84.140",
    "147.32.84.130",
    "147.32.84.160",
    "10.0.2.15",
    "192.168.106.141",
    "192.168.106.131",
    "172.16.253.130",
    "172.16.253.131",
    "172.16.253.129",
    "172.16.253.240",
    "74.78.117.238",
    "158.65.110.24",
    "192.168.3.35",
    "192.168.3.25",
    "192.168.3.65",
    "172.29.0.116",
    "172.29.0.109",
    "172.16.253.132",
    "192.168.248.165",
    "10.37.130.4",
    "131.202.243.84", 
    "198.164.30.2", "192.168.5.122", 
    "192.168.5.122", 
    "192.168.5.122", "192.168.5.122", "192.168.5.122", 
    "192.168.2.110", 
    "192.168.4.120", "192.168.1.103",
    "192.168.2.113", "192.168.4.118" , "192.168.2.109", "192.168.2.105", "192.168.5.122"]

In [ ]:
Test.add_column(gl.SArray(final),'Predicted_Label')

In [ ]:

truePosIPs = Test[(Test['isBot']==1) & (Test['Predicted_Label']==1)]['Source'].unique()
i=0
for x in truePosIPs:
    if x in iplist:
        print x
        i=i+1
    else:
        print "Not in Malicious IP List " + x
print i



In [ ]:
falseNegIPs = Test[(Test['isBot']==1) & (Test['Predicted_Label']==0)]['Source'].unique()
len(falseNegIPs)

i=0
for x in falseNegIPs:
    if x in iplist:
        print x
        i=i+1
    else:
        print "Not in Malicious IP List " + x
print i

BotsInTrain = SF[(SF['isBot']==1)]['Source'].unique()

i=0
for x in falseNegIPs:
    if x in BotsInTrain:
        print x + " in both train and test, still not detected"
    else:
        i=i+1
print str(i) + " Bot IPs present in test, not in train, undetected "

i=0
for x in truePosIPs:
    if x in BotsInTrain:
        print x + " in both train and test, detected"
    else:
        i=i+1
print str(i) + " Bot IPs present in test, not in train, still detected => Novelty Detection "

### 22-10-2016

In [1]:
import graphlab as gl
import tensorflow as tf
SF = gl.SFrame.read_csv('ISCX_Botnet-Testing_Ports_Only_Sorted_Flow_IAT_Null.csv',verbose=False)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1477113345.log


This non-commercial license of GraphLab Create for academic use is assigned to himadri.mishra.cse13@iitbhu.ac.in and will expire on September 21, 2017.


In [1]:
import graphlab as gl
tp = gl.SFrame.read_csv('ISCX_Botnet-Testing.csv')

This non-commercial license of GraphLab Create for academic use is assigned to himadri.mishra.cse13@iitbhu.ac.in and will expire on September 21, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1477323264.log


Finished parsing file /home/os-net/kartik-himadri-BTP/Dataset/ISCX_Botnet-Testing.csv

Parsing completed. Parsed 100 lines in 0.986556 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,float,str,str,str,int,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

Read 1471970 lines. Lines per second: 212693

Read 2878577 lines. Lines per second: 225595

Read 4058710 lines. Lines per second: 227091

Finished parsing file /home/os-net/kartik-himadri-BTP/Dataset/ISCX_Botnet-Testing.csv

Parsing completed. Parsed 5114514 lines in 22.441 secs.

In [2]:
tp = tp[(tp['Source Port']=='')|(tp['Destination Port']=='')]
for x in sorted(tp['Protocol'].unique()):
    print x,len(tp[tp['Protocol']==x])

0x0700 4684
0x3339 1
0x4000 83
0x4b0d 1
0xe000 1
0xffff 2773
ARP 188815
Ethernet 1
ICMP 7423
ICMPv6 1686
IGMPv1 4
IGMPv2 30
IGMPv3 1010
IPv4 140010
IPv6 48
LLC 140
LLDP 5684
STP 85272
TDMoP 529


In [3]:
tp = tp[(tp['Source Port']=='')]
for x in sorted(tp['Protocol'].unique()):
    print x,len(tp[tp['Protocol']==x])

0x0700 4684
0x3339 1
0x4000 83
0x4b0d 1
0xe000 1
0xffff 2773
ARP 188815
Ethernet 1
ICMP 7423
ICMPv6 1686
IGMPv1 4
IGMPv2 30
IGMPv3 1010
IPv4 140010
IPv6 48
LLC 140
LLDP 5684
STP 85272
TDMoP 529


In [4]:
tp = tp[(tp['Destination Port']=='')]
for x in sorted(tp['Protocol'].unique()):
    print x,len(tp[tp['Protocol']==x])

0x0700 4684
0x3339 1
0x4000 83
0x4b0d 1
0xe000 1
0xffff 2773
ARP 188815
Ethernet 1
ICMP 7423
ICMPv6 1686
IGMPv1 4
IGMPv2 30
IGMPv3 1010
IPv4 140010
IPv6 48
LLC 140
LLDP 5684
STP 85272
TDMoP 529


In [5]:
len(tp)

438195

In [8]:
for x in sorted(tp['Protocol'].unique()):
    print len(tp[tp['Protocol']==x]),',',

4684 , 1 , 83 , 1 , 1 , 2773 , 188815 , 1 , 7423 , 1686 , 4 , 30 , 1010 , 140010 , 48 , 140 , 5684 , 85272 , 529 ,
